# Step 2
### LinkedIn Profile's Link Extraction

In [4]:
import requests
from bs4 import BeautifulSoup

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope and credentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)

# Authorize the client
client = gspread.authorize(creds)

# Open the Google Sheet
sheet = client.open('calendar event info').GetEvents

# Assuming your emails are in the first column
email_column = sheet.col_values(4)

for email_address in email_column:
    # Split the email address into first name, last name, and company (assumed that all emails are in this format: firstname.lastname@company.com)
    first_name, last_name, company = email_address.split('@')[0].split('.')


# Function to retrieve the first Google search result link for a given query
def get_first_google_search_link(query):
    
    # Set user agent to mimic a web browser
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    # URL for Google search and parameters including the search query
    google_search_url = 'https://www.google.com/search'
    params = {'q': query}
    
    # Sending a GET request to Google Search
    response = requests.get(google_search_url, headers=headers, params=params)
    
    # Check if the response is successful (status code 200)
    if response.status_code == 200:
        # Parse the response content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all links (anchor tags) in the search results
        search_results = soup.find_all('a', href=True)
        
        # Iterate through each search result link
        for result in search_results:
            href = result['href']  # Get the href attribute (URL)
            
            # Check if the link leads to a LinkedIn profile
            if "https://www.linkedin.com/in" in href:
                return href  # Return the URL if it is a LinkedIn profile link
    
    # Return None if no suitable LinkedIn profile link is found
    return None


search_string = f"{first_name} {last_name} {company} LinkedIn"
first_link = get_first_google_search_link(search_string)

# Extract the URL part
start = first_link.find("url=") + 4
end = first_link.find("&", start)
extracted_linkedin_url = first_link[start:end]

print(extracted_linkedin_url)

https://www.linkedin.com/in/srikaran-reddy
